<a href="https://colab.research.google.com/github/elliot-brooks/nlu-coursework/blob/main/src/av_model_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [80]:
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
import nltk
import re

nltk.download('punkt')

FIRST_PAIR = 0
SECOND_PAIR = 1

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Load training data

In [86]:
training_corpus = pd.read_csv("train.csv", encoding='utf-8')
# pairwise_labels = np.array(training_corpus['label'])
# pairwise_data = np.array(training_corpus[['text_1', 'text_2']])

# Pre-process training data

In [90]:
def preprocess(string):
  string = str(string)
  tokens = word_tokenize(string)
  tokens = [token.lower() for token in tokens]
  output = ' '.join(tokens)
  #   separated_string = re.sub(r'([^\w\s])', r' \1 ', str(string))
  return output

training_corpus["text_1"] = training_corpus["text_1"].apply(lambda x: preprocess(x))
training_corpus["text_2"] = training_corpus["text_2"].apply(lambda x: preprocess(x))

pairwise_labels = np.array(training_corpus['label'])
# Trying to get to work for 1 text (needs updating for 2)
pairwise_data = training_corpus['text_1'].values.tolist()

["nick ( kevin anderson ) goes back to his hometown to take care of his dying mother ( kim novak ) . there he encounters an old college buddy ( bill pullman ) and his beautiful wife ( pamela gidley ) . he also gets involved with a 40 year old sex murder that may have something to do with him . . . i caught this in a theatre back in 1991 . it was part of the boston film festival and i had heard it had some incredibly beautiful cinematography . well - - it does . it just does n't have much of a story to go with it . also the film moves so slowly - - i literally fell asleep ! seeing it again 14 years later , i fell asleep again ! this film is dull and slow . it 's one of those art films full of strange characters who act oddly and mutter obscure dialogue ( especially the police", "kate nelligan , always a forthright and gripping actress , gets to show many different sides as a single mother in new york city whose little boy disappears one day while walking a short distance to school ; she

In [106]:
tk = Tokenizer(filters='/t/n', oov_token='UNK', lower=True)
tk.fit_on_texts(pairwise_data)

In [113]:
print(len(tk.word_index))
if 'a' in tk.word_index:
  print("True")
print(list(tk.word_index.keys())[100:1000])
print(pairwise_data[0])

64033
True
['eve', 'da', 'would', 'ime', ';', 'good', 're', 'vi', 'lo', 'see', 'ar', 'she', 'la', 'more', 'which', 'mo', 'si', 'wo', '...', 'corp', 'very', '&', 'had', "'m", 'de', 'gs', 'ew', 'please', 'forwarded', 'kay', 'ks', 'bee', 'heir', 'well', 'urlli', 'hough', 'oo', 'ds', 'really', 'ory', 'par', 'did', 'also', 'him', 'our', 'ch', 'ever', 'fu', 'ees', 'day', 'af', 'cha', 'how', 'were', 'firs', 'hem', 'does', 'di', 'ers', 'mi', 'j', 'joh', '$', 'much', 'ki', 'direc', 'grea', 'ai', 'people', 'could', 'ex', 'mos', 'ive', 'way', 'us', 'ski', 'ure', '=20', 'because', 'make', 'back', 'goi', 'sa', 'work', 'ake', 'ic', '2', '01', 'should', 'ill', 'chris', "'ve", 'ely', 'bo', 'may', 'over', 'bes', 'ces', 'hese', 'las', 'ies', 'igh', 'able', 'ess', 'dow', 'ey', 'where', 'eed', 'ur', 'fa', 'off', "'ll", 'bei', 'while', 'eres', 'ry', 'mee', 'america', 'charac', 'made', 'ci', '12', 'c', 'ia', 'oday', 'agai', 'der', 'kami', 'old', 'call', 'op', 'give', '10', '_developme', 'agreeme', 'age', 'l

# Define Language Model

# Train Model

# Save Model